In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").getOrCreate()

# 重複データの扱いについて

# 以下データには 3 つの問題がある
# ID が 3 になっている行が 2 つあり、その内容がまったく同じ
# ID が 1 と 4 の行データが同じ
# ID が 5 の行が 2 つある、がデータは違う
df = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 167.2, 5.4, 45, 'M'),
    (3, 124.1, 5.2, 23, 'F'),
    (4, 144.5, 5.9, 33, 'M'),
    (5, 133.2, 5.7, 54, 'F'),
    (3, 124.1, 5.2, 23, 'F'),
    (5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])
df.show()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/16 10:33:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/16 10:33:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [1]:
print('Count of rows: {}'.format(df.count()))
print('Count of distinct rows: {}'.format(df.distinct().count()))

NameError: name 'df' is not defined

In [8]:
# 全ての値が一致した重複行を取り除く
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [10]:
print('Count of ids: {}'.format(df.count()))
# id 列以外のデータで重複行のカウント
print('Count of distinct ids: {}'.format(df.select([column for column in df.columns if column != 'id']).distinct().count()))

Count of ids: 6
Count of distinct ids: 5


In [12]:
df = df.dropDuplicates(subset=[column for column in df.columns if column != 'id'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [15]:
# 重複した ID の確認
import pyspark.sql.functions as fn

df.agg(
    fn.count('id').alias('count'),
    fn.countDistinct('id').alias('distinct'),
).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



In [16]:
# 各行に単調増加するユニークな ID の割り振り
# monotonically_increasing_id() は
# 80億レコード以下であればユニークが担保される
df.withColumn('new_id', fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+

